# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
import pandas as pd
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.dropna(inplace = True) 
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lalmohan,22.3384,90.7419,81.45,70,5,8.14,BD,1619889557
1,lorengau,-2.0226,147.2712,79.83,81,72,2.80,PG,1619889557
2,yellowknife,62.4560,-114.3525,25.00,63,75,13.80,CA,1619889557
3,cherskiy,68.7500,161.3000,-2.87,99,33,2.77,RU,1619889558
4,namibe,-15.1961,12.1522,76.87,74,4,9.48,AO,1619889558


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph. Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

# Use boolean indexing to filter the weather_df dataframe
narrowed_cities_df = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0), :]
narrowed_cities_df = narrowed_cities_df.dropna(how='any')
narrowed_cities_df.reset_index(inplace=True)
narrowed_cities_df
del narrowed_cities_df['index']
narrowed_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint-philippe,-21.3585,55.7679,77.00,78,0,3.44,RE,1619889583
1,saint-pierre,-21.3393,55.4781,77.00,78,0,3.44,RE,1619889584
2,port keats,-14.2500,129.5500,75.16,54,0,5.50,AU,1619889598
3,qesarya,32.5189,34.9046,72.00,78,0,2.30,IL,1619889610
4,istanbul,41.0351,28.9833,75.20,64,0,2.30,TR,1619889686


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(narrowed_cities_df, columns=['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Hotel Name'])
hotel_df

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,saint-philippe,RE,-21.3585,55.7679,77.00,NaN
1,saint-pierre,RE,-21.3393,55.4781,77.00,NaN
2,port keats,AU,-14.2500,129.5500,75.16,NaN
3,qesarya,IL,32.5189,34.9046,72.00,NaN
4,istanbul,TR,41.0351,28.9833,75.20,NaN
5,saint-joseph,RE,-21.3667,55.6167,77.00,NaN
6,balykshi,KZ,47.0667,51.8667,71.60,NaN
7,tacuarembo,UY,-31.7333,-55.9833,77.59,NaN
8,linqiong,CN,30.4159,103.4609,75.20,NaN


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [9]:
# Save Data to csv
hotel_df.to_csv("../Output_Data/Hotel_Output.csv", encoding="utf-8", index=False)

# Visualize to confirm airport data appears
hotel_df.head(10)

,City,Country,Lat,Lng,Max Temp,Hotel Name,Hotel Address,Hotel Rating
0,saint-philippe,RE,-21.3585,55.7679,77.00,Les Embruns Du Baril,Réunion,3.8
1,saint-pierre,RE,-21.3393,55.4781,77.00,Le Battant Des Lames,117 Boulevard Hubert Delisle,4.2
2,port keats,AU,-14.2500,129.5500,75.16,TDC Village (Hundred Man Camp),"20 Kanarlda St, Wadeye",3.9
3,qesarya,IL,32.5189,34.9046,72.00,Dan Caesarea Resort - Happiness Is Around,"Rothschild St 1, Caesarea",4.4
4,istanbul,TR,41.0351,28.9833,75.20,"Radisson Blu Bosphorus Hotel, Istanbul","Ortaköy, Çırağan Cd. No:46, Beşiktaş",4.3
5,saint-joseph,RE,-21.3667,55.6167,77.00,"""Plantation Bed and Breakfast","124 Route de Jean Petit, Saint Joseph Ile de l...",4.8
6,balykshi,KZ,47.0667,51.8667,71.60,Hotel Atyrau,"улица Сатпаева 6, Atyrau",4.3
7,tacuarembo,UY,-31.7333,-55.9833,77.59,Hotel y Museo La Cumparsita,Brigadier Gral. Fructuoso Rivera km 383,4.5
8,linqiong,CN,30.4159,103.4609,75.20,Lanting Hotel,"335 Feihong Rd, Qionglai City, Chengdu",5.0


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,-21.3585,55.7679
1,-21.3393,55.4781
2,-14.2500,129.5500
3,32.5189,34.9046
4,41.0351,28.9833
5,-21.3667,55.6167
6,47.0667,51.8667
7,-31.7333,-55.9833
8,30.4159,103.4609


In [11]:
# Add marker layer ontop of heat map
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Display figure
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…